In [9]:
import numpy as np
import pandas as pd
import theano
import theano.tensor as T

import lasagne

In [10]:
path="data/"
data=pd.read_csv(path+"input_train.csv", index_col="ID").filter(regex= "EEG.*")
labels=pd.read_csv(path+"challenge_output_data_training_file_sleep_stages_classification.csv", sep=";")

In [11]:
def build_cnn(input_var=None):
    network = lasagne.layers.InputLayer(shape=(40, 3750),
                                        input_var=input_var)
    network = lasagne.layers.Conv1DLayer(
        network, num_filters=16, filter_size=1,
        nonlinearity=lasagne.nonlinearities.rectify,
        #W=lasagne.init.GlorotUniform())
        )
    network = lasagne.layers.MaxPool1DLayer(network, pool_size=5)
    #network = lasagne.layers.Conv2DLayer(
    #    network, num_filters=32, filter_size=(10, 1),
    #    nonlinearity=lasagne.nonlinearities.rectify,
    #    W=lasagne.init.GlorotUniform())
    #network = lasagne.layers.MaxPool2DLayer(network, pool_size=(5, 1))
    network = lasagne.layers.DenseLayer(
        lasagne.layers.dropout(network, p=.5),
        num_units=1000,
        nonlinearity=lasagne.nonlinearities.rectify)
    network = lasagne.layers.DenseLayer(
        lasagne.layers.dropout(network, p=.5),
        num_units=5,
        nonlinearity=lasagne.nonlinearities.softmax)

    return network

In [7]:
%pdb

Automatic pdb calling has been turned OFF


In [8]:
# Load the dataset
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.values, labels["TARGET"].values, test_size=0.10, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=42)
# Prepare Theano variables for inputs and targets
input_var = T.matrix('inputs')
target_var = T.ivector('targets')
# Create neural network model
network = build_cnn(input_var)

IndexError: tuple index out of range

In [6]:
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()

/home/bat/anaconda2/lib/python2.7/site-packages/lasagne/layers/conv.py:489: UserWarning: The `image_shape` keyword argument to `tensor.nnet.conv2d` is deprecated, it has been renamed to `input_shape`.
  border_mode=border_mode)


In [7]:
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(
        loss, params, learning_rate=0.01, momentum=0.9)

In [8]:
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                        target_var)
test_loss = test_loss.mean()

In [9]:
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                  dtype=theano.config.floatX)

In [10]:
from tempfile import mkstemp

In [11]:
#tempfile.mkstemp(dir="../tmp")

In [12]:
#import os
#os.environ["THEANO_FLAGS"] = "blas.ldflags=-llapack -lblas -lgfortran"

In [13]:
train_fn =theano.function([input_var, target_var], loss, updates=updates)

In [14]:
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

In [15]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]



In [16]:
import time
num_epochs = 40
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, 40, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, 40, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

TypeError: ('Bad input argument to theano function with name "<ipython-input-13-e75be2a3211c>:1"  at index 0(0-based)', 'Wrong number of dimensions: expected 4, got 2 with shape (40, 3750).')

In [17]:
X_train

array([[ -3.27090272,  -5.17772412,  -5.93906008, ...,  -4.48844825,
         -6.07158702,  -6.32548659],
       [  0.81069452,   0.47279711,   0.20199726, ..., -36.21900083,
        -34.24005984, -33.34316947],
       [-21.6893264 , -19.19169591, -18.05410131, ..., -31.74934691,
        -32.01672954, -31.93865097],
       ..., 
       [ 23.2823655 ,  22.67404351,  21.55815244, ...,  17.25645074,
         14.23969712,   9.22338637],
       [ -9.58074305,  -6.7713347 ,  -6.12195638, ...,   7.8565183 ,
         11.19261089,  13.00891586],
       [ -0.92509361,  -0.31970086,   0.12211495, ...,   0.88637613,
          0.5123374 ,  -0.10109219]])

In [ ]:
predicted_values = test_prediction.flatten()
print "test loss:", test_loss